*TP réalisé par Rachel Blin dans le cadre du cours d'Alexandrina Rogozan*

# Codage cyclique

Pour ce TP, on considère un code correcteur d'erreur cyclique par blocs ayant ayant des mots-code de longueur 7 bits dans lesquels on retrouve m=3 bits de parité. Les n=7 bits constituant un mot-code sont considérés comme coefficients d'un polynôme de degré n-1 soit :  

c(x) = d0 + d1x + d2x² + ... + d6x^6    

Le polynôme générateur utilisé pour obtenir un mot-code est le suivant : 

g(x) = x^3 + x + 1

Afin de retrouver tous les mots-code constituant ce code cyclique il suffit de multiplier les k bits d'info avec k = n-m par le polynôme générateur.

## Ecriture des mots-code 

1) Dans un premier temps, déterminer toutes les valeurs possibles des bits d'info

In [3]:
import numpy as np

n = 7
m = 3

def values_bits_info(m, n):
    """ Function that returns all the possibles values for the information bits
    
    # Arguments:
        - m: The number of parity bits
        - n: The length of the codeword
    
    # Returns:
        A list of all the possible combinations of the information bits    
    """
    # retourne un polynôme avec les bits correspondants au plus grand degré à gauche
    bits_info = []
    for i in range(pow(2, n-m)):
        bit_temp = np.zeros(n-m, dtype=int)
        bit_str = bin(i)[2:]
        for i in range(len(bit_str)):
            bit_temp[len(bit_temp)-i-1] = int(bit_str[len(bit_str)-1-i])
        bits_info.append(bit_temp)
    return bits_info

bits_info = values_bits_info(m, n)
for bit in bits_info:
    print(bit)

[0 0 0 0]
[0 0 0 1]
[0 0 1 0]
[0 0 1 1]
[0 1 0 0]
[0 1 0 1]
[0 1 1 0]
[0 1 1 1]
[1 0 0 0]
[1 0 0 1]
[1 0 1 0]
[1 0 1 1]
[1 1 0 0]
[1 1 0 1]
[1 1 1 0]
[1 1 1 1]


2) Ecrivez maintenant tous les mots-code de ce code cyclique.

In [5]:
g = np.array([1, 0, 1, 1])

def polynomial_multiplication(x, y):
    """ A function to multiply two binary polynoms
    
    # Arguments:
        - x: A binary polynom
        - y: A binary polynom
        
    # Returns:
        A binary polynom
    """
    # prend en entrée deux poynômes avec les bits correspondants au plus grand degré à gauche
    # retourne un polynôme avec les bits correspondants au plus grand degré à droite
    y = np.flip(y)
    x = np.flip(x)
    polynom = np.zeros(x.shape[0]+y.shape[0]-1, dtype=int)
    for i in range(x.shape[0]):
        pol_temp = y*x[i]
        for j in range(i, pol_temp.shape[0]+i):
            polynom[j] = (polynom[j] + pol_temp[j-i]) % 2
    return polynom
        

def cyclic_codeword(bits_info, g):
    """ A function that computes all the possible cyclic codewords
    
    # Arguments:
        - bits_info: A list containing all the possibles information bits combination
        - g: The generator polynom
    
    """
    # retourne des polynôme avec les bits correspondants au plus grand degré à droite
    codewords = []
    for bit in bits_info:
        word = polynomial_multiplication(bit, g)
        codewords.append(word)
    return mots_code

codewords = cyclic_codeword(bits_info, g)
for word in codewords:
    print(word)

[0 0 0 0 0 0 0]
[1 1 0 1 0 0 0]
[0 1 1 0 1 0 0]
[1 0 1 1 1 0 0]
[0 0 1 1 0 1 0]
[1 1 1 0 0 1 0]
[0 1 0 1 1 1 0]
[1 0 0 0 1 1 0]
[0 0 0 1 1 0 1]
[1 1 0 0 1 0 1]
[0 1 1 1 0 0 1]
[1 0 1 0 0 0 1]
[0 0 1 0 1 1 1]
[1 1 1 1 1 1 1]
[0 1 0 0 0 1 1]
[1 0 0 1 0 1 1]


On dit qu'un code est cyclique parce que les mots-code de celui-ci possèdent un motif récurrent (sans compter les mots-code dont tous les bits sont à 0 ou à 1) et qui peut être inversé. 

3) Quel est le motif de ce code cyclique?

Le motif de ce code cyclique est "__1101__" qui peut être inversé en "__1011__". On le retrouve systématiquement dans les mots-code dont tous les bits ne sont ni égaux à 0 ni à 1 :  

0 0 0 0 0 0 0  
__1 1 0 1__ 0 0 0  
0 __1 1 0 1__ 0 0  
__1 0 1 1__ 1 0 0  
0 0 __1 1 0 1__ 0  
__1 1__ 1 0 0 __1 0__  
0 __1 0 1 1__ 1 0  
__1</t>__ 0 0 0 __1 1 0__  
0 0 0 __1 1 0 1__  
**1</t>** 1 0 0 **1 0 1**  
**0 1 1** 1 0 0 **1</t>**  
**1 0 1** 0 0 0 **1</t>**  
0 0 __1 0 1 1__ 1  
1 1 1 1 1 1 1  
**0 1** 0 0 0 **1 1**  
1 0 0 __1 0 1 1__ 

## Calcul de la distance minimale de ce code

Pour rappel, la distance de Hamming entre deux séquences de même longueur est le nombre de caractères qui différent entre ces deux séquences.

__Exemple__ :  
La distance de Hamming entre "*1000101*" et "*1100101*" est de 1.  
La distance de Hamming entre "*1000101*" et "*0000111*" est de 2.  

4) Calculez la distance minimale de Hamming pour ce code. Pour celà, il est plus judicieux de calculer la distance de Hamming entre le mot-code dont tous les bits sont à zéros et les autres mots_codes et de retenir la distance minimale.

In [8]:
def Hamming_distance(codewords):
    """ This function computes the minimal Hamming thistance for the codewords
    
    # Argument:
        - codewords: A list containing all the codewords
        
    # Returns:
        The minimal Hamming distance of these codewords
    """
    comparison_word = codewords[0]
    H_distance = codewords[0].shape[0]
    for word in codewords[1:]:
        distance_Hamming_temp = 0
        for i in range(word.shape[0]):
            distance_Hamming_temp += (comparison_word[i] + word[i]) % 2
        if distance_Hamming_temp < H_distance:
            H_distance = distance_Hamming_temp
    return H_distance

d_Hamming = Hamming_distance(codewords)
print("La distance de Hamming de ce code cyslique est : ", d_Hamming)

La distance de Hamming de ce code cyslique est :  3


Le nombre maximal d'erreurs corrigibles par ce code est déterminé par le plus grand entier strictement inférieur à la distance minimale de Hamming / 2.  

5) calculez le nombre maximal d'erreurs corrigibles par ce code

In [11]:
def number_errors(d_Hamming):
    """ A function that computes the number of errors that this cyclic code can correct
    
    # Argument:
        - d_Hamming: The minimal Hamming distance
        
    # Returns:
        The number of errors this code can correct
    """
    nb_errors = d_Hamming // 2
    return nb_errors

nb_errors = number_errors(d_Hamming)
print("Le nombre d'erreurs corrigibles par ce code est : ", nb_errors)

Le nombre d'erreurs corrigibles par ce code est :  1


## Calcul des mots-code sous forme systématique 

Pour rappel, dans notre cas de figure on a m = 3 bits de parité.    

Le polynôme des bits de parité (p(x)) est le reste de la division du polynome formé à partir des bits d'information (i(x)) multiplié par x^m par g(x) soit :  
p(x) = reste((i(x) * x^m) / g(x))

Ces bits de parité permettent d'écrire les mots-code sous forme systématique avec d0, d1 et d2 les coefficients du polynôme des bits de parité p(x) et d3, d4, d5 et d6 les coefficents du polynôme d'information i(x).    

6) Ecrivez les mots-code sous forme systématique.

In [12]:
def polynomial_soustraction(x, y):
    """ A function that computes the binary polynomial soustraction of two polynoms
    
    # Arguments:
        - x: A binary polynom
        - y: A binary polynom
        
    # Returns:
        A binary polynom   
    """
    
    # prend en entrée deux poynômes avec les bits correspondants au plus grand degré à droite
    # retourne un polynôme avec les bits correspondants au plus grand degré à droite
    if x.shape[0] > y.shape[0]:
        complement = x.shape[0] - y.shape[0]
        for c in range(complement):
            y = np.append(y,0)
    if x.shape[0] < y.shape[0]:
        complement = y.shape[0] - x.shape[0]
        for c in range(complement):
            x = np.append(x,0)
    polynom = np.zeros(x.shape[0], dtype=int)
    for i in range(x.shape[0]):
        polynom[i] = (x[i] - y[i]) % 2
    return polynom

def rest_division(x, y):
    """ A function that computes the rest of the binary division
    
    # Arguments:
        - x: A binary polynom
        - y: A binary polynom
        
    # Returns:
        A binary polynom
    """
    # prend en entrée deux poynômes avec les bits correspondants au plus grand degré à gauche
    # retourne un polynôme avec les bits correspondants au plus grand degré à droite
    x = np.flip(x).copy()
    y = np.flip(y).copy()
    coeff_poly_multiplicateur = 1
    while coeff_poly_multiplicateur > 0:
        i_plus_grand_coeff_x = 0
        i_plus_grand_coeff_y = 0
        for i in range(x.shape[0]):
            if x[i] == 1:
                i_plus_grand_coeff_x = i
        for j in range(y.shape[0]):
            if y[j] == 1:
                i_plus_grand_coeff_y = j
        coeff_poly_multiplicateur = i_plus_grand_coeff_x-i_plus_grand_coeff_y
        if coeff_poly_multiplicateur >= 0:
            poly_multiplicateur = np.zeros(coeff_poly_multiplicateur+1, dtype=int)
            poly_multiplicateur[-1] = 1
            y_temp = polynomial_multiplication(np.flip(poly_multiplicateur), np.flip(y))
            x = polynomial_soustraction(x, y_temp)
        y = np.flip(y).copy()
    return x

def poly_x_m(m):
    """ A function that creates the polynom x^m
    
    # Arguments:
        - m: The degree of the desired polynom
    
    # Returns:
        A binary polynom
    """
    # retourne un polynôme avec les bits correspondants au plus grand degré à gauche 
    poly = np.zeros(m+1, dtype=int)
    poly[0] = 1
    return poly

def bits_info_multi_x_m(bits_info, x):
    """ A function that multiplies the polynom corresponding to the information bits with the polynom x^m
    
    # Arguments:
        - bits_info: A list containing all the information bits combinasions
        - x: The polynom x^m
        
    # Returns:
        A list containing all the information bits polynoms multiplied by x^m
    """
    # retourne des polynômes avec les bits correspondants au plus grand degré à droite
    bits_info_multi = []
    for bit in bits_info:
        bits_info_temp = polynomial_multiplication(bit, x)
        bits_info_multi.append(bits_info_temp)
    return bits_info_multi

def compute_parity_bits(bits_info_multi, g):
    """ A function that computes the parity bits of the codewords
    
    # Arguments:
        - bits_info_multi: A list containing all the polynoms corresponding to the information bits multiplied by x^m
        - g: The generator polynom
        
    # Returns:
        A list containing all the parity bits corresponding to each codeword
    """
    # retourne des polynômes avec les bits correspondants au plus grand degré à droite
    bits_parite = []
    for bit in bits_info_multi:
        bits_parite.append(rest_division(np.flip(bit), g))
    return bits_parite

def systematic_form(bits_info, bits_parite, m):
    """ A function that computes the systematic form of the cyclic codewords
    
    # Arguments:
        - bits_info: A list containing all the information bits of the cyclic codewords
        - bits_parite: A list contaning all the parity bits of the cyclic codewords
        - m: The number of parity bits
    
    """
    # retourne des polynômes avec les bits correspondants au plus grand degré à droite
    i = 0
    code_systematique = bits_parite
    for bit in bits_info:
        code_systematique[i][m:] = np.flip(bit).copy()
        i += 1
    return code_systematique
    

poly = poly_x_m(m)
bits_info_multi = bits_info_multi_x_m(bits_info, poly)
for i in bits_info_multi:
    print(i)
print("\n")    
bits_parite = compute_parity_bits(bits_info_multi, g)
for j in bits_parite:
    print(j)
print("\n")    
code_systematique = systematic_form(bits_info, bits_parite, m)
for c in code_systematique:
    print(c)

[0 0 0 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 1 1 0 0]
[0 0 0 0 0 1 0]
[0 0 0 1 0 1 0]
[0 0 0 0 1 1 0]
[0 0 0 1 1 1 0]
[0 0 0 0 0 0 1]
[0 0 0 1 0 0 1]
[0 0 0 0 1 0 1]
[0 0 0 1 1 0 1]
[0 0 0 0 0 1 1]
[0 0 0 1 0 1 1]
[0 0 0 0 1 1 1]
[0 0 0 1 1 1 1]


[0 0 0 0 0 0 0]
[1 1 0 0 0 0 0]
[0 1 1 0 0 0 0]
[1 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 1 1 0 0 0 0]
[1 0 1 0 0 0 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[1 0 1 0 0 0 0]
[0 0 0 0 0 0 0]
[1 1 1 0 0 0 0]
[0 0 1 0 0 0 0]
[1 1 1 0 0 0 0]
[0 1 0 0 0 0 0]


[0 0 0 0 0 0 0]
[1 1 0 1 0 0 0]
[0 1 1 0 1 0 0]
[1 1 0 1 1 0 0]
[1 0 0 0 0 1 0]
[0 0 1 1 0 1 0]
[0 1 1 0 1 1 0]
[1 0 1 1 1 1 0]
[0 1 0 0 0 0 1]
[1 0 0 1 0 0 1]
[1 0 1 0 1 0 1]
[0 0 0 1 1 0 1]
[1 1 1 0 0 1 1]
[0 0 1 1 0 1 1]
[1 1 1 0 1 1 1]
[0 1 0 1 1 1 1]


## Simulation d'erreurs

Dans les questions précédentes, on a calculé le nombre d'erreurs corrigibles par ce code cyclique. Afin de savoir si le message reçu contient une erreur, on calcule le syndrome s(x) qui est le reste du polynôme correspondant à la forme systématique (r(x)) par le polynôme générateur (g(x)), pour résumer :  

s(x) = reste(r(x)/g(x))  

Si s(x) est égal à 0, cela signifie que le message est reçu sans erreur, en revanche s'il est différent de 0 cela veut dire que le message reçu comporte une erreur.    

Pour illustrer cet exemple, on va prendre le la forme systématique du message correspondant à 2, soit "*0 1 1 0 1 0 0*". Dans un premier temps on va simuler une erreur sur le bit d0, dans un deuxième temps deux erreurs, une sur le bit d0 et une autre sur le bit d1 et enfin trois erreurs une sur le bit d0, une sur le bit d2 et une sur le bit d3.

7) Calculez le syndrome pour le message sans erreurs, puis pour le message avec une seule erreur, pour celui comportant deux erreurs et enfin pour celui comportant 3 erreurs. Que peut-on en conclure?

In [14]:
mess = code_systematique[2].copy()

mess0 = np.flip(mess).copy()

mess1 = mess.copy()
mess1[0] = (mess1[0] + 1) % 2
mess1 = np.flip(mess1).copy()

mess2 = mess.copy()
mess2[0] = (mess2[0] + 1) % 2
mess2[1] = (mess2[1] + 1) % 2
mess2 = np.flip(mess2).copy()

mess3 = mess.copy()
mess3[0] = (mess3[0] + 1) % 2
mess3[2] = (mess3[1] + 1) % 2
mess3[3] = (mess3[2] + 1) % 2
mess3 = np.flip(mess3).copy()

print(mess0)
print(mess1)
print(mess2)
s0 = rest_division(mess0, g)
print("Pour un message sans erreur, s(x) = ", s0)
s1 = rest_division(mess1, g)
print("Pour un message avec une erreur, s(x) = ", s1)
s2 = rest_division(mess2, g)
print("Pour un message avec 2 erreurs, s(x) = ", s2)
s3 = rest_division(mess3, g)
print("Pour un message avec 3 erreurs, s(x) = ", s3)

[0 0 1 0 1 1 0]
[0 0 1 0 1 1 1]
[0 0 1 0 1 0 1]
Pour un message sans erreur, s(x) =  [0 0 0 0 0 0 0]
Pour un message avec une erreur, s(x) =  [1 0 0 0 0 0 0]
Pour un message avec 2 erreurs, s(x) =  [1 1 0 0 0 0 0]
Pour un message avec 3 erreurs, s(x) =  [0 0 0 0 0 0 0]


On constate que lorsque l'on simule 3 erreurs, le syndrôme est égal à zéro. En effet, le syndrôme dans ce cas résultant de la division d'un polynôme de degré 6 par un polynôme de degré 3, il sera maximum un polynôme de degré 2, ce qui donne 2^3 = 8 syndromes différents. Parmis ces syndrômes, il y en a 7 pathologiques (s(x)=0 signifie qu'il n'y a pas d'erreur) qui correspondent chacun à un code d'erreur pour chacun des 7 bits du message. Au-dela de 2 erreurs, on ne peut donc plus savoir où est l'erreur dans le message même si le syndrôme est différent de 0.